In [3]:
from scipy.special import xlogy
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [55]:
    def sigmoid(x):
        return 1/(1+np.exp(-x))
    
    def softmax(x):
        """Compute softmax values for each sets of scores in x."""
        return np.exp(x) / np.sum(np.exp(x), axis=0) 

    def relu(x):
        return x*(x>0)
    
    def tanh(x):
        return (np.exp(2*x)-1)/(np.exp(2*x)+1)
    
    def binaryCrossEntropy(trueOutputs, Predictions):
        error = xlogy(trueOutputs, Predictions+ 1e-15) + xlogy(1 - trueOutputs, 1 - Predictions + 1e-15)
        error = -np.sum(error)
        return error
        
    def derivativeActivation(activation, valuesToChange):
        if activation == 'sigmoid':
            return valuesToChange*(1-valuesToChange)
        elif activation == 'tanh':
            return 1 - valuesToChange**2
        elif activation == 'relu':
            return np.ones(valuesToChange.shape)*(valuesToChange>=0)

In [182]:
class neural_network:
    layers = 0
    parameters = {}
    activations = {'sigmoid': sigmoid, 'relu': relu, 'softmax': softmax, 'tanh': tanh}
    
    def showParams(self, node=0):
        if node == 0:
            for i in range(self.layers):
                temp = self.parameters['layer'+str(i+1)]
                print("\nLayer "+str(i+1)+":\t"+str(temp['noOfNodes'])+ " Nodes")
                print("Activation Function: ",temp['activation'])
                print("Weights = ",temp['weights'])
                print("Bias = ", temp['bias'])
        else:
            temp = self.parameters['layer'+str(node)]
            print("\nLayer "+str(node)+":\t"+str(temp['noOfNodes'])+ " Nodes")
            print("Activation Function: ",temp['activation'])
            print("Weights = ",temp['weights'])
            print("Bias = ", temp['bias'])
            
    def info(self):
        print("Number of Layers:\t", self.layers)
        sumParams = 0
        for i in range(self.layers):
            temp = self.parameters['layer'+str(i+1)]
            print("\nLayer "+str(i+1)+":\t"+str(temp['noOfNodes'])+ " Nodes")
            print("Activation Function: ",temp['activation'])
            sumParams += temp['weights'].shape[0]*temp['weights'].shape[1]
            sumParams += temp['bias'].shape[0]*temp['bias'].shape[1]
        print("\nTotal number of parameters: ", sumParams)
    
    def predict(self, inputValues):
        cache = {}
        for i in range(self.layers):
            layer = self.parameters['layer'+str(i+1)]
            
            if i == 0:
                z = layer['weights'].dot(inputValues) + layer['bias']
                a = self.activations[layer['activation']](z)
                cache['layer'+str(i+1)] = {'z':z, 'a':a}
            else:
                z = layer['weights'].dot(cache['layer'+str(i)]['a']) + layer['bias']
                a = self.activations[layer['activation']](z)
                cache['layer'+str(i+1)] = {'z':z, 'a':a}
            
        return cache['layer'+str(self.layers)]['a']
    
    def forward(self, inputValues):
        cache = {}
        for i in range(self.layers):
            layer = self.parameters['layer'+str(i+1)]
            
            if i == 0:
                z = layer['weights'].dot(inputValues) + layer['bias']
                a = self.activations[layer['activation']](z)
                cache['layer'+str(i+1)] = {'z':z, 'a':a}
            else:
                z = layer['weights'].dot(cache['layer'+str(i)]['a']) + layer['bias']
                a = self.activations[layer['activation']](z)
                cache['layer'+str(i+1)] = {'z':z, 'a':a}
            
        return cache['layer'+str(self.layers)]['a'], cache
    
    def removeLayer(self):
        value = self.parameters.pop('layer'+str(self.layers))
        print("Removed Layer "+str(self.layers)+ ' with values : ', value)
        self.layers -= 1
    

    def addLayer(self, noOfNodes, activationFunction, **kwargs):
        self.layers += 1
        
        if self.layers == 1:
            if len(kwargs.items()) != 0:
                for key, value in kwargs.items():
                    if key == 'inputSize':
                        self.parameters['layer'+str(self.layers)] = {'weights':np.random.randn(noOfNodes, value), 'bias' : np.zeros((noOfNodes,1)), 'activation':activationFunction, 'noOfNodes':noOfNodes}
            else:
                print("ERROR: Cannot Compile Model, Input Size not given\nTry Giving an argument example: inputSize = 5")
                
        else:
           self.parameters['layer'+str(self.layers)] = {'weights':np.random.randn(noOfNodes, self.parameters['layer'+str(self.layers-1)]['noOfNodes']), 'bias' : np.zeros((noOfNodes,1)), 'activation':activationFunction, 'noOfNodes':noOfNodes}
        
        print("Added a layer with "+str(noOfNodes)+" nodes, and "+activationFunction+" Activation Function.")
        
    def backprop(self, inputs, outputs, epochs = 20, learning_rate = 0.2):
        cache = {}
        for i in range(1, epochs+1):
            predictions, forward_cache = self.forward(inputs)
            print("Epoch " + str(i) + " ||")
            loss = binaryCrossEntropy(outputs, predictions)
            print(loss)
            for j in range(self.layers, 0, -1):
                if j == self.layers:
                    da = -outputs/(predictions+ 1e-15) + ((1-outputs)/(1-predictions+ 1e-15))
                    dz = da * derivativeActivation(self.parameters['layer'+str(j)]['activation'], forward_cache['layer'+str(j)]['a'])
                   
                    dw = (dz.dot(forward_cache['layer'+str(j-1)]['a'].T))/inputs.shape[1]
                    db = np.sum(dz, axis = 1, keepdims = True)/inputs.shape[1]
                    da_1 = self.parameters['layer'+str(j)]['weights'].T.dot(dz)
                    cache['layer'+str(j)] = {'da':da, 'dz':dz, 'dw':dw, 'db':db}
                    cache['layer'+str(j-1)] = {'da':da_1}
                else:
                    if j > 1:
                        dz = cache['layer'+str(j)]['da']*derivativeActivation(self.parameters['layer'+str(j)]['activation'], forward_cache['layer'+str(j)]['a'])
                        dw = (dz.dot(forward_cache['layer'+str(j-1)]['a'].T))/inputs.shape[1]
                        db = np.sum(dz, axis = 1, keepdims = True)/inputs.shape[1]
                        da_1 = self.parameters['layer'+str(j)]['weights'].T.dot(dz)
                        cache['layer'+str(j)].update({'dz':dz, 'dw':dw, 'db':db})
                        cache['layer'+str(j-1)] = {'da':da_1}
                    else:
                        dz = cache['layer'+str(j)]['da']*derivativeActivation(self.parameters['layer'+str(j)]['activation'], forward_cache['layer'+str(j)]['a'])
                        dw = (dz.dot(inputs.T))/inputs.shape[1]
                        db = np.sum(dz, axis = 1, keepdims = True)/inputs.shape[1]
                        da_1 = self.parameters['layer'+str(j)]['weights'].T.dot(dz)
                        cache['layer'+str(j)].update({'dz':dz, 'dw':dw, 'db':db})
                    
            for k in range(1, self.layers+1):
                layer = self.parameters['layer'+str(k)]
                layer['weights'] -= learning_rate*cache['layer'+str(k)]['dw']
                layer['bias'] -= learning_rate*cache['layer'+str(k)]['db']

In [183]:
x = np.array([[3, -5, 0, 8, 9, 10, 0, 0, 10, 5], [4, 15, 1, 10, 0, 0, 0, 100, 20, 0]])
y = np.array([[1, 1, 0, 1, 0, 0, 1, 0, 1, 0]])

In [184]:
model = neural_network()
model.addLayer(24, 'sigmoid', inputSize = 2)
model.addLayer(24, 'sigmoid')
model.addLayer(1, 'sigmoid')
model.showParams()

Added a layer with 24 nodes, and sigmoid Activation Function.
Added a layer with 24 nodes, and sigmoid Activation Function.
Added a layer with 1 nodes, and sigmoid Activation Function.

Layer 1:	24 Nodes
Activation Function:  sigmoid
Weights =  [[ 5.48771406e-01  9.55447777e-01]
 [ 7.66387456e-02 -1.48196745e+00]
 [-5.58074791e-01  2.38227435e-01]
 [-1.02686589e+00 -5.29896778e-01]
 [ 5.28638956e-01 -3.23714499e-01]
 [ 1.36194973e+00  2.25030179e+00]
 [ 1.02917534e-01 -8.80997889e-01]
 [-5.87852717e-01  7.68895632e-01]
 [ 3.94096770e-01 -1.03676131e+00]
 [ 4.15607011e-01  6.68318089e-01]
 [ 9.37109078e-01 -7.21401744e-01]
 [ 1.02746987e+00 -9.11424540e-01]
 [-1.54055470e+00  2.00038885e-03]
 [ 9.90767125e-01  8.45568772e-01]
 [ 3.62824163e-01 -1.25779063e+00]
 [ 5.66566625e-01  7.70109824e-01]
 [ 1.63565922e+00  2.17000757e+00]
 [-7.80314353e-01 -3.59714722e-01]
 [ 6.03532939e-03 -1.88048118e+00]
 [ 9.54097182e-01 -5.33685225e-01]
 [ 1.38045974e+00 -3.98813438e-02]
 [-7.56711670e-01 -1

In [185]:
model.forward(x)

(array([[0.85490378, 0.95277023, 0.92584728, 0.80955417, 0.88570429,
         0.88555451, 0.95508944, 0.90690471, 0.85009523, 0.89637163]]),
 {'layer1': {'z': array([[ 5.46810533e+00,  1.15878596e+01,  9.55447777e-01,
            1.39446490e+01,  4.93894266e+00,  5.48771406e+00,
            0.00000000e+00,  9.55447777e+01,  2.45966696e+01,
            2.74385703e+00],
          [-5.69795356e+00, -2.26127055e+01, -1.48196745e+00,
           -1.42065645e+01,  6.89748710e-01,  7.66387456e-01,
            0.00000000e+00, -1.48196745e+02, -2.88729615e+01,
            3.83193728e-01],
          [-7.21314634e-01,  6.36378548e+00,  2.38227435e-01,
           -2.08232398e+00, -5.02267312e+00, -5.58074791e+00,
            0.00000000e+00,  2.38227435e+01, -8.16199212e-01,
           -2.79037396e+00],
          [-5.20018479e+00, -2.81412220e+00, -5.29896778e-01,
           -1.35138949e+01, -9.24179304e+00, -1.02686589e+01,
            0.00000000e+00, -5.29896778e+01, -2.08665945e+01,
           -5

In [186]:
model.backprop(x, y, 2000, 0.2)

Epoch 1 ||
12.204103748875406
Epoch 2 ||
7.635739921549277
Epoch 3 ||
6.0185370326660985
Epoch 4 ||
5.197515341602755
Epoch 5 ||
4.708479150467741
Epoch 6 ||
4.411197846983079
Epoch 7 ||
4.211975273463013
Epoch 8 ||
4.0648293310738595
Epoch 9 ||
3.9491019096655284
Epoch 10 ||
3.8538785702223923
Epoch 11 ||
3.7727321890458683
Epoch 12 ||
3.701629888046419
Epoch 13 ||
3.6379144139295354
Epoch 14 ||
3.579751652798896
Epoch 15 ||
3.5258136425803177
Epoch 16 ||
3.4750892789367787
Epoch 17 ||
3.4267687495845705
Epoch 18 ||
3.3801766304245997
Epoch 19 ||
3.3347472051508795
Epoch 20 ||
3.290047803613076
Epoch 21 ||
3.245853453442118
Epoch 22 ||
3.202238616122625
Epoch 23 ||
3.1595855420510133
Epoch 24 ||
3.1184165845765515
Epoch 25 ||
3.0791302697892586
Epoch 26 ||
3.0418538008232514
Epoch 27 ||
3.0064862249552045
Epoch 28 ||
2.972820927484157
Epoch 29 ||
2.9406398968113545
Epoch 30 ||
2.9097564559530342
Epoch 31 ||
2.880024276626121
Epoch 32 ||
2.851331795339911
Epoch 33 ||
2.823592840224094


Epoch 476 ||
0.3311384347043174
Epoch 477 ||
0.33013554991799116
Epoch 478 ||
0.3291375404881591
Epoch 479 ||
0.3281443756191868
Epoch 480 ||
0.32715602474554595
Epoch 481 ||
0.3261724575298788
Epoch 482 ||
0.325193643861081
Epoch 483 ||
0.3242195538524016
Epoch 484 ||
0.3232501578395624
Epoch 485 ||
0.32228542637889235
Epoch 486 ||
0.32132533024548254
Epoch 487 ||
0.3203698404313568
Epoch 488 ||
0.3194189281436575
Epoch 489 ||
0.318472564802853
Epoch 490 ||
0.31753072204095495
Epoch 491 ||
0.31659337169975876
Epoch 492 ||
0.3156604858290967
Epoch 493 ||
0.31473203668510746
Epoch 494 ||
0.3138079967285221
Epoch 495 ||
0.31288833862296517
Epoch 496 ||
0.3119730352332729
Epoch 497 ||
0.31106205962382516
Epoch 498 ||
0.3101553850568943
Epoch 499 ||
0.30925298499100606
Epoch 500 ||
0.30835483307931977
Epoch 501 ||
0.30746090316801983
Epoch 502 ||
0.30657116929472283
Epoch 503 ||
0.30568560568690134
Epoch 504 ||
0.304804186760317
Epoch 505 ||
0.30392688711747373
Epoch 506 ||
0.3030536815460

Epoch 737 ||
0.17468556336544844
Epoch 738 ||
0.17433935648317242
Epoch 739 ||
0.17399432375632445
Epoch 740 ||
0.1736504597285229
Epoch 741 ||
0.1733077589749171
Epoch 742 ||
0.1729662161019717
Epoch 743 ||
0.17262582574725804
Epoch 744 ||
0.17228658257923885
Epoch 745 ||
0.17194848129706278
Epoch 746 ||
0.1716115166303532
Epoch 747 ||
0.171275683339006
Epoch 748 ||
0.17094097621298077
Epoch 749 ||
0.1706073900721029
Epoch 750 ||
0.17027491976585793
Epoch 751 ||
0.16994356017319395
Epoch 752 ||
0.16961330620232334
Epoch 753 ||
0.16928415279052475
Epoch 754 ||
0.16895609490394917
Epoch 755 ||
0.16862912753742446
Epoch 756 ||
0.16830324571426517
Epoch 757 ||
0.16797844448608076
Epoch 758 ||
0.16765471893258446
Epoch 759 ||
0.16733206416140906
Epoch 760 ||
0.16701047530791582
Epoch 761 ||
0.16668994753501343
Epoch 762 ||
0.1663704760329722
Epoch 763 ||
0.1660520560192412
Epoch 764 ||
0.16573468273826855
Epoch 765 ||
0.1654183514613226
Epoch 766 ||
0.16510305748630977
Epoch 767 ||
0.16478

0.10973604015387027
Epoch 1017 ||
0.10958183907325036
Epoch 1018 ||
0.10942802003132532
Epoch 1019 ||
0.10927458170073322
Epoch 1020 ||
0.10912152275997238
Epoch 1021 ||
0.10896884189337078
Epoch 1022 ||
0.10881653779105328
Epoch 1023 ||
0.10866460914891413
Epoch 1024 ||
0.10851305466858352
Epoch 1025 ||
0.10836187305739844
Epoch 1026 ||
0.10821106302837284
Epoch 1027 ||
0.10806062330016715
Epoch 1028 ||
0.10791055259705823
Epoch 1029 ||
0.10776084964891045
Epoch 1030 ||
0.10761151319114563
Epoch 1031 ||
0.10746254196471547
Epoch 1032 ||
0.10731393471606913
Epoch 1033 ||
0.10716569019712804
Epoch 1034 ||
0.10701780716525514
Epoch 1035 ||
0.1068702843832259
Epoch 1036 ||
0.10672312061920143
Epoch 1037 ||
0.10657631464670032
Epoch 1038 ||
0.10642986524456914
Epoch 1039 ||
0.10628377119695569
Epoch 1040 ||
0.10613803129328074
Epoch 1041 ||
0.10599264432821076
Epoch 1042 ||
0.10584760910163218
Epoch 1043 ||
0.10570292441862092
Epoch 1044 ||
0.10555858908941729
Epoch 1045 ||
0.1054146019294

Epoch 1268 ||
0.08022129490527972
Epoch 1269 ||
0.08013293854169448
Epoch 1270 ||
0.08004475630844392
Epoch 1271 ||
0.07995674771907113
Epoch 1272 ||
0.07986891228886275
Epoch 1273 ||
0.07978124953483981
Epoch 1274 ||
0.07969375897575179
Epoch 1275 ||
0.07960644013206992
Epoch 1276 ||
0.07951929252597587
Epoch 1277 ||
0.07943231568135979
Epoch 1278 ||
0.07934550912380794
Epoch 1279 ||
0.07925887238059895
Epoch 1280 ||
0.07917240498069349
Epoch 1281 ||
0.07908610645472995
Epoch 1282 ||
0.07899997633501471
Epoch 1283 ||
0.07891401415551705
Epoch 1284 ||
0.07882821945186015
Epoch 1285 ||
0.07874259176131504
Epoch 1286 ||
0.07865713062279307
Epoch 1287 ||
0.07857183557683949
Epoch 1288 ||
0.07848670616562552
Epoch 1289 ||
0.07840174193294251
Epoch 1290 ||
0.07831694242419275
Epoch 1291 ||
0.07823230718638555
Epoch 1292 ||
0.07814783576812884
Epoch 1293 ||
0.07806352771962186
Epoch 1294 ||
0.0779793825926489
Epoch 1295 ||
0.07789539994057317
Epoch 1296 ||
0.07781157931832755
Epoch 1297 ||
0

Epoch 1536 ||
0.06152629845682479
Epoch 1537 ||
0.06147148242811662
Epoch 1538 ||
0.061416754665242025
Epoch 1539 ||
0.06136211496530974
Epoch 1540 ||
0.061307563126030085
Epoch 1541 ||
0.061253098945713534
Epoch 1542 ||
0.06119872222326617
Epoch 1543 ||
0.06114443275819196
Epoch 1544 ||
0.06109023035058578
Epoch 1545 ||
0.061036114801134946
Epoch 1546 ||
0.060982085911114155
Epoch 1547 ||
0.060928143482385455
Epoch 1548 ||
0.060874287317395324
Epoch 1549 ||
0.060820517219173
Epoch 1550 ||
0.06076683299132858
Epoch 1551 ||
0.06071323443804858
Epoch 1552 ||
0.06065972136409738
Epoch 1553 ||
0.060606293574812516
Epoch 1554 ||
0.06055295087610474
Epoch 1555 ||
0.06049969307445353
Epoch 1556 ||
0.06044651997690717
Epoch 1557 ||
0.0603934313910801
Epoch 1558 ||
0.06034042712514969
Epoch 1559 ||
0.06028750698785593
Epoch 1560 ||
0.060234670788498444
Epoch 1561 ||
0.06018191833693556
Epoch 1562 ||
0.06012924944357998
Epoch 1563 ||
0.06007666391939938
Epoch 1564 ||
0.06002416157591329
Epoch 15

Epoch 1794 ||
0.049827675491676854
Epoch 1795 ||
0.049790285707758966
Epoch 1796 ||
0.04975294697076632
Epoch 1797 ||
0.04971565918075553
Epoch 1798 ||
0.0496784222380374
Epoch 1799 ||
0.04964123604317433
Epoch 1800 ||
0.04960410049698108
Epoch 1801 ||
0.04956701550052419
Epoch 1802 ||
0.04952998095511876
Epoch 1803 ||
0.049492996762330245
Epoch 1804 ||
0.04945606282397349
Epoch 1805 ||
0.049419179042111044
Epoch 1806 ||
0.0493823453190509
Epoch 1807 ||
0.049345561557349975
Epoch 1808 ||
0.04930882765980899
Epoch 1809 ||
0.04927214352947462
Epoch 1810 ||
0.04923550906963743
Epoch 1811 ||
0.04919892418383089
Epoch 1812 ||
0.04916238877583153
Epoch 1813 ||
0.049125902749658175
Epoch 1814 ||
0.04908946600957013
Epoch 1815 ||
0.04905307846006727
Epoch 1816 ||
0.04901674000588925
Epoch 1817 ||
0.048980450552014596
Epoch 1818 ||
0.048944210003660414
Epoch 1819 ||
0.04890801826628047
Epoch 1820 ||
0.04887187524556559
Epoch 1821 ||
0.04883578084744399
Epoch 1822 ||
0.048799734978076825
Epoch 1

In [ ]:
model.predict

In [187]:
model.predict(x)

array([[0.99413155, 0.99629167, 0.01165433, 0.99891748, 0.00157557,
        0.0015654 , 0.99017641, 0.0036787 , 0.99850303, 0.00243847]])

In [195]:
model.predict([[50], [10]])

array([[0.00242251]])

In [196]:
binaryCrossEntropy(y, model.predict(x))

0.04304746872488264

In [95]:
from sklearn.datasets import load_breast_cancer

In [152]:
dataset = load_breast_cancer()
x = dataset.data[:200]
x = x.T
x.shape
y = dataset.target[:200]
y = y.reshape((1, 200))
y.shape
x.shape

(30, 200)

In [167]:
breast = neural_network()

In [168]:
breast.addLayer(5, 'sigmoid', inputSize = 30)

Added a layer with 5 nodes, and sigmoid Activation Function.


In [169]:
breast.addLayer(10, 'sigmoid')

Added a layer with 10 nodes, and sigmoid Activation Function.


In [170]:
breast.addLayer(10, 'sigmoid')

Added a layer with 10 nodes, and sigmoid Activation Function.


In [171]:
breast.addLayer(1, 'sigmoid')

Added a layer with 1 nodes, and sigmoid Activation Function.


In [172]:
breast.info()

Number of Layers:	 4

Layer 1:	5 Nodes
Activation Function:  sigmoid

Layer 2:	10 Nodes
Activation Function:  sigmoid

Layer 3:	10 Nodes
Activation Function:  sigmoid

Layer 4:	1 Nodes
Activation Function:  sigmoid

Total number of parameters:  336


In [181]:
breast.backprop(x, y, 100, learning_rate=2)

<ipython-input-55-22fa955bf275>:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


Epoch 1 ||
134.69308424843695
[[ 1.17758286  2.3731553   2.3731553   2.3731553   2.3731553   2.3731553
   2.3731553   2.3731553   2.3731553   2.3731553   2.3731553   2.3731553
   2.3731553   2.3731553   2.3731553   2.3731553   2.3731553   2.3731553
   1.17758286 -1.72824975 -1.72824975 -1.72824975  2.3731553   1.17758286
   1.17758286  2.3731553   2.3731553   2.3731553   1.17758286  2.3731553
   2.3731553   1.17758286  2.3731553   2.3731553   2.3731553   2.3731553
   2.3731553  -1.72824975  2.3731553   2.3731553   2.3731553   2.3731553
   2.3731553   2.37315529  2.3731553   2.3731553  -1.72824975  2.3731553
  -1.72824975 -1.72824975 -1.72824975 -1.72824975 -1.72824975  2.3731553
   2.3731553  -1.72824975  1.17758286  2.3731553  -1.72824975 -1.72824975
  -1.72824975 -1.72824975  2.3731553  -1.72824975  1.17758286  2.3731553
  -1.72824975 -1.72824975 -1.72824975 -1.72824975  2.3731553  -1.72824975
   1.17758286  2.3731553  -1.72824975  2.3731553  -1.72824975  2.3731553
   2.3731553  -1.7

[[ 1.0924432   2.01002797  2.01002797  2.01002797  2.01002797  2.01002797
   2.01002797  2.01002797  2.01002797  2.01002797  2.01002797  2.01002797
   2.01002797  2.01002797  2.01002797  2.01002797  2.01002797  2.01002797
   1.0924432  -1.99007159 -1.99007159 -1.99007159  2.01002797  1.0924432
   1.0924432   2.01002797  2.01002797  2.01002797  1.0924432   2.01002797
   2.01002797  1.0924432   2.01002797  2.01002797  2.01002797  2.01002797
   2.01002797 -1.99007159  2.01002797  2.01002797  2.01002797  2.01002797
   2.01002797  2.01002797  2.01002797  2.01002797 -1.99007159  2.01002797
  -1.99007159 -1.99007159 -1.99007159 -1.99007159 -1.99007159  2.01002797
   2.01002797 -1.99007159  1.0924432   2.01002797 -1.99007159 -1.99007159
  -1.99007159 -1.99007159  2.01002797 -1.99007159  1.0924432   2.01002797
  -1.99007159 -1.99007159 -1.99007159 -1.99007159  2.01002797 -1.99007159
   1.0924432   2.01002797 -1.99007159  2.01002797 -1.99007159  2.01002797
   2.01002797 -1.99007159 -1.99007159 -

Epoch 36 ||
131.7718614872603
[[ 1.07385964  2.01463687  2.01463687  2.01463687  2.01463687  2.01463687
   2.01463687  2.01463687  2.01463687  2.01463687  2.01463687  2.01463687
   2.01463687  2.01463687  2.01463687  2.01463687  2.01463687  2.01463687
   1.07385964 -1.98557427 -1.98557427 -1.98557427  2.01463687  1.07385964
   1.07385964  2.01463687  2.01463687  2.01463687  1.07385964  2.01463687
   2.01463687  1.07385964  2.01463687  2.01463687  2.01463687  2.01463687
   2.01463687 -1.98557427  2.01463687  2.01463687  2.01463687  2.01463687
   2.01463687  2.01463687  2.01463687  2.01463687 -1.98557427  2.01463687
  -1.98557427 -1.98557427 -1.98557427 -1.98557427 -1.98557427  2.01463687
   2.01463687 -1.98557427  1.07385964  2.01463687 -1.98557427 -1.98557427
  -1.98557427 -1.98557427  2.01463687 -1.98557427  1.07385964  2.01463687
  -1.98557427 -1.98557427 -1.98557427 -1.98557427  2.01463687 -1.98557427
   1.07385964  2.01463687 -1.98557427  2.01463687 -1.98557427  2.01463687
   2.014

   2.01813671  2.01813671]]
[[ 0.05656182  0.50449343  0.50449343  0.50449343  0.50449343  0.50449343
   0.50449343  0.50449343  0.50449343  0.50449343  0.50449343  0.50449343
   0.50449343  0.50449343  0.50449343  0.50449343  0.50449343  0.50449343
   0.05656182 -0.49550657 -0.49550657 -0.49550657  0.50449343  0.05656182
   0.05656182  0.50449343  0.50449343  0.50449343  0.05656182  0.50449343
   0.50449343  0.05656182  0.50449343  0.50449343  0.50449343  0.50449343
   0.50449343 -0.49550657  0.50449343  0.50449343  0.50449343  0.50449343
   0.50449343  0.50449343  0.50449343  0.50449343 -0.49550657  0.50449343
  -0.49550657 -0.49550657 -0.49550657 -0.49550657 -0.49550657  0.50449343
   0.50449343 -0.49550657  0.05656182  0.50449343 -0.49550657 -0.49550657
  -0.49550657 -0.49550657  0.50449343 -0.49550657  0.05656182  0.50449343
  -0.49550657 -0.49550657 -0.49550657 -0.49550657  0.50449343 -0.49550657
   0.05656182  0.50449343 -0.49550657  0.50449343 -0.49550657  0.50449343
   0.50449

Epoch 75 ||
131.51765797847207
[[ 1.04971094  2.02071254  2.02071254  2.02071254  2.02071254  2.02071254
   2.02071254  2.02071254  2.02071254  2.02071254  2.02071254  2.02071254
   2.02071254  2.02071254  2.02071254  2.02071254  2.02071254  2.02071254
   1.04971094 -1.97970777 -1.97970777 -1.97970777  2.02071254  1.04971094
   1.04971094  2.02071254  2.02071254  2.02071254  1.04971094  2.02071254
   2.02071254  1.04971094  2.02071254  2.02071254  2.02071254  2.02071254
   2.02071254 -1.97970777  2.02071254  2.02071254  2.02071254  2.02071254
   2.02071254  2.02071254  2.02071254  2.02071254 -1.97970777  2.02071254
  -1.97970777 -1.97970777 -1.97970777 -1.97970777 -1.97970777  2.02071254
   2.02071254 -1.97970777  1.04971094  2.02071254 -1.97970777 -1.97970777
  -1.97970777 -1.97970777  2.02071254 -1.97970777  1.04971094  2.02071254
  -1.97970777 -1.97970777 -1.97970777 -1.97970777  2.02071254 -1.97970777
   1.04971094  2.02071254 -1.97970777  2.02071254 -1.97970777  2.02071254
   2.02

[[ 1.04287241  2.02242053  2.02242053  2.02242053  2.02242053  2.02242053
   2.02242053  2.02242053  2.02242053  2.02242053  2.02242053  2.02242053
   2.02242053  2.02242053  2.02242053  2.02242053  2.02242053  2.02242053
   1.04287241 -1.97807113 -1.97807113 -1.97807113  2.02242053  1.04287241
   1.04287241  2.02242053  2.02242053  2.02242053  1.04287241  2.02242053
   2.02242053  1.04287241  2.02242053  2.02242053  2.02242053  2.02242053
   2.02242053 -1.97807113  2.02242053  2.02242053  2.02242053  2.02242053
   2.02242053  2.02242053  2.02242053  2.02242053 -1.97807113  2.02242053
  -1.97807113 -1.97807113 -1.97807113 -1.97807113 -1.97807113  2.02242053
   2.02242053 -1.97807113  1.04287241  2.02242053 -1.97807113 -1.97807113
  -1.97807113 -1.97807113  2.02242053 -1.97807113  1.04287241  2.02242053
  -1.97807113 -1.97807113 -1.97807113 -1.97807113  2.02242053 -1.97807113
   1.04287241  2.02242053 -1.97807113  2.02242053 -1.97807113  2.02242053
   2.02242053 -1.97807113 -1.97807113 